In [1]:
import pymongo
import os
import pandas as pd

import time

## Addresses IP des 3 instances:

In [2]:
DB_IP = ["172.31.31.100", "172.31.31.101", "172.31.31.102"]

## Nom du replica set:

In [3]:
REPLICA_SET = "replica-set"

## Liste des états:

2016-11-08-20-00_Minnesota.txt
2016-11-08-20-01_Alabama.txt
2016-11-08-20-01_Montana.txt
2016-11-08-20-01_Utah.txt
2016-11-08-20-02_Caroline_du_Sud.txt
2016-11-08-20-02_New_York.txt
2016-11-08-20-03_Nouveau_Mexique.txt
2016-11-08-20-04_Californie.txt
2016-11-08-20-04_Colorado.txt
2016-11-08-20-05_Connecticut.txt
2016-11-08-20-08_District_de_Columbia.txt
2016-11-08-20-08_Kentucky.txt
2016-11-08-20-09_Caroline_du_Nord.txt
2016-11-08-20-09_Vermont.txt
2016-11-08-20-10_Dakota_du_Sud.txt
2016-11-08-20-11_Alaska.txt
2016-11-08-20-13_Arkansas.txt
2016-11-08-20-13_Washington.txt
2016-11-08-20-15_Hawai.txt
2016-11-08-20-17_Wisconsin.txt
2016-11-08-20-19_Georgie.txt
2016-11-08-20-22_Ohio.txt
2016-11-08-20-24_Missouri.txt
2016-11-08-20-24_New_Hampshire.txt
2016-11-08-20-24_Virginie.txt
2016-11-08-20-27_Iowa.txt
2016-11-08-20-28_Delaware.txt
2016-11-08-20-29_Arizona.txt
2016-11-08-20-31_Louisiane.txt
2016-11-08-20-31_Massachusetts.txt
2016-11-08-20-31_Texas.txt
2016-11-08-20-34_Dakota_du_Nord.txt
2016-11-08-20-35_Tennessee.txt
2016-11-08-20-36_Wyoming.txt
2016-11-08-20-37_Floride.txt
2016-11-08-20-37_Illinois.txt
2016-11-08-20-40_Nevada.txt
2016-11-08-20-41_Idaho.txt
2016-11-08-20-41_Pennsylvanie.txt
2016-11-08-20-43_Virginie_Occidentale.txt
2016-11-08-20-48_Maryland.txt
2016-11-08-20-48_Mississippi.txt
2016-11-08-20-48_Rhode_Island.txt
2016-11-08-20-50_Michigan.txt
2016-11-08-20-51_Oklahoma.txt
2016-11-08-20-52_Nebraska.txt
2016-11-08-20-56_New_Jersey.txt
2016-11-08-20-57_Indiana.txt
2016-11-08-20-57_Kansas.txt
2016-11-08-20-58_Maine.txt
2016-11-08-20-59_Oregon.txt"

In [4]:
MONGO_CLI = "mongodb://{}:27017,{}:27017,{}:27017/?replicaSet={}".format(DB_IP[0], DB_IP[1], DB_IP[2], REPLICA_SET)

In [5]:
REPLICA = "{}/{}:27017,{}:27017,{}:27017".format(REPLICA_SET, DB_IP[0], DB_IP[1], DB_IP[2])
print(REPLICA)

replica-set/172.31.31.100:27017,172.31.31.101:27017,172.31.31.102:27017


In [6]:
MONGO_CLI

'mongodb://172.31.31.100:27017,172.31.31.101:27017,172.31.31.102:27017/?replicaSet=replica-set'

In [7]:
c = pymongo.MongoClient(MONGO_CLI)

In [8]:
c.database_names()

['local', 'dontexist', 'POTUS']

##### Fonction appel script sur le front

In [9]:
import subprocess
from subprocess import call
def chargement(filename,DIR,col):
        #DIR = "~/POTUS/data_POTUS/"
        # on a mis les scripts sur le même repertoire que les données
        script = DIR + "import.bash"
        #col = "col1"
        # print script
        call([script, filename ,col])
        # insertion dans la collection l' etat traité
        c.POTUS.traitement.insert_one({'traitement': filename})
        print("Traitement : ", filename)
        
        

In [ ]:
etat = 'Minnesota'
myCurso = c.POTUS.col1.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
x =[]
for i in myCurso:
    x.append(i)

In [34]:

def prep_dico(nom_etat,x):

    #dic_election_list =[]
    n_autres = 0
    n_trump_vote = 0
    n_clinton_vote = 0
    for elt in x :
        
        f = elt['total']
        e  = elt['_id']
        if (e['STATE'] == nom_etat) :
            if (e['CANDIDATE'] == 'Trump') : 
                n_trump_vote = f
            else : 
                if e['CANDIDATE'] == 'Clinton' :
                    n_clinton_vote = f
                else : 
                    n_autres += f
    print('STATE     : ' + str(nom_etat))
    print('n_trum    : ' + str(n_trump_vote))
    print('n_clinton : ' + str(n_clinton_vote))    
    print('autres    : ' + str(n_autres))
    
    
    #    li = [e['STATE'],e['CANDIDATE'],f]
    #    dic_election_list.append(li)

    #df =pd.DataFrame(dic_election_list, columns=["STATE", "CANDIDATE", "ToTaL"])
    c.POTUS.synthese.insert_one(
    {
     "state": nom_etat,
     "Clinton": n_clinton_vote,
     "Trump" : n_trump_vote,
     "Other" : n_autres   })


In [10]:
list_File = [
'2016-11-08-20-00_Minnesota.txt'
,'2016-11-08-20-01_Alabama.txt'
,'2016-11-08-20-01_Montana.txt'
,'2016-11-08-20-01_Utah.txt'
,'2016-11-08-20-02_Caroline_du_Sud.txt'
,'2016-11-08-20-02_New_York.txt'
,'2016-11-08-20-03_Nouveau_Mexique.txt'
,'2016-11-08-20-04_Californie.txt'
,'2016-11-08-20-04_Colorado.txt'
,'2016-11-08-20-05_Connecticut.txt'
,'2016-11-08-20-08_District_de_Columbia.txt'
,'2016-11-08-20-08_Kentucky.txt'
,'2016-11-08-20-09_Caroline_du_Nord.txt'
,'2016-11-08-20-09_Vermont.txt'
,'2016-11-08-20-10_Dakota_du_Sud.txt'
,'2016-11-08-20-11_Alaska.txt'
,'2016-11-08-20-13_Arkansas.txt'
,'2016-11-08-20-13_Washington.txt'
,'2016-11-08-20-15_Hawai.txt'
,'2016-11-08-20-17_Wisconsin.txt'
,'2016-11-08-20-19_Georgie.txt'
,'2016-11-08-20-22_Ohio.txt'
,'2016-11-08-20-24_Missouri.txt'
,'2016-11-08-20-24_New_Hampshire.txt'
,'2016-11-08-20-24_Virginie.txt'
,'2016-11-08-20-27_Iowa.txt'
,'2016-11-08-20-28_Delaware.txt'
,'2016-11-08-20-29_Arizona.txt'
,'2016-11-08-20-31_Louisiane.txt'
,'2016-11-08-20-31_Massachusetts.txt'
,'2016-11-08-20-31_Texas.txt'
,'2016-11-08-20-34_Dakota_du_Nord.txt'
,'2016-11-08-20-35_Tennessee.txt'
,'2016-11-08-20-36_Wyoming.txt'
,'2016-11-08-20-37_Floride.txt'
,'2016-11-08-20-37_Illinois.txt'
,'2016-11-08-20-40_Nevada.txt'
,'2016-11-08-20-41_Idaho.txt'
,'2016-11-08-20-41_Pennsylvanie.txt'
,'2016-11-08-20-43_Virginie_Occidentale.txt'
,'2016-11-08-20-48_Maryland.txt'
,'2016-11-08-20-48_Mississippi.txt'
,'2016-11-08-20-48_Rhode_Island.txt'
,'2016-11-08-20-50_Michigan.txt'
,'2016-11-08-20-51_Oklahoma.txt'
,'2016-11-08-20-52_Nebraska.txt'
,'2016-11-08-20-56_New_Jersey.txt'
,'2016-11-08-20-57_Indiana.txt'
,'2016-11-08-20-57_Kansas.txt'
,'2016-11-08-20-58_Maine.txt'
,'2016-11-08-20-59_Oregon.txt']

In [24]:
list_File_col1 = list_File[0:7]
print(list_File_col1)


['2016-11-08-20-00_Minnesota.txt', '2016-11-08-20-01_Alabama.txt', '2016-11-08-20-01_Montana.txt', '2016-11-08-20-01_Utah.txt', '2016-11-08-20-02_Caroline_du_Sud.txt', '2016-11-08-20-02_New_York.txt', '2016-11-08-20-03_Nouveau_Mexique.txt']


# Alimentation Collection 1

In [25]:
%%time
DIR = "/home/ec2-user/POTUS/data_POTUS/"
for file in list_File_col1:
    filename = DIR + file
    chargement(filename,DIR,"col1")
    myCurso = c.POTUS.col1.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)    

Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-00_Minnesota.txt
Minnesota
n_trum : 1322951
n_clinton1367716
autres :254146
STATE : Minnesota
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-01_Alabama.txt
Alabama
n_trum : 1318255
n_clinton729547
autres :89680
STATE : Alabama
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-01_Montana.txt
Montana
n_trum : 279240
n_clinton177709
autres :59952
STATE : Montana
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-01_Utah.txt
Utah
n_trum : 515231
n_clinton310676
autres :305523
STATE : Utah
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-02_Caroline_du_Sud.txt
Caroline_du_Sud
n_trum : 1155389
n_clinton855373
autres :92265
STATE : Caroline_du_Sud
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-02_New_York.txt
New_York
n_trum : 2814346
n_clinton4547418
autres :425300
STATE : New_York
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-03_Nouveau_Mexique.txt
Nouveau_Mexiqu

## Alimentation Collection 2

In [26]:
list_File_col2= list_File[7]
print (list_File_col2)

2016-11-08-20-04_Californie.txt


In [27]:
%%time
DIR = "/home/ec2-user/POTUS/data_POTUS/"
filename = DIR + list_File_col2
print (filename)
chargement(filename,DIR,"col2")
myCurso = c.POTUS.col2.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
x =[]
for i in myCurso:
    x.append(i)
etat = list_File_col2[17:-4]
print (etat)
prep_dico(etat,x)    

/home/ec2-user/POTUS/data_POTUS/2016-11-08-20-04_Californie.txt
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-04_Californie.txt
Californie
n_trum : 4483810
n_clinton8753788
autres :1372901
STATE : Californie
CPU times: user 220 ms, sys: 40 ms, total: 260 ms
Wall time: 11min 5s


## Alimentation Collection 3

In [28]:
list_File_col3 =list_File[8:18]
print (list_File_col3)

['2016-11-08-20-04_Colorado.txt', '2016-11-08-20-05_Connecticut.txt', '2016-11-08-20-08_District_de_Columbia.txt', '2016-11-08-20-08_Kentucky.txt', '2016-11-08-20-09_Caroline_du_Nord.txt', '2016-11-08-20-09_Vermont.txt', '2016-11-08-20-10_Dakota_du_Sud.txt', '2016-11-08-20-11_Alaska.txt', '2016-11-08-20-13_Arkansas.txt', '2016-11-08-20-13_Washington.txt']


In [29]:
%%time
for file in list_File_col3:
    filename = DIR + file
    chargement(filename,DIR,"col3")
    myCurso = c.POTUS.col3.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)  

Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-04_Colorado.txt
Colorado
n_trum : 1202484
n_clinton1338870
autres :317862
STATE : Colorado
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-05_Connecticut.txt
Connecticut
n_trum : 673215
n_clinton897572
autres :105147
STATE : Connecticut
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-08_District_de_Columbia.txt
District_de_Columbia
n_trum : 12723
n_clinton282830
autres :77022
STATE : District_de_Columbia
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-08_Kentucky.txt
Kentucky
n_trum : 1202971
n_clinton628854
autres :117429
STATE : Kentucky
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-09_Caroline_du_Nord.txt
Caroline_du_Nord
n_trum : 2362631
n_clinton2189316
autres :217693
STATE : Caroline_du_Nord
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-09_Vermont.txt
Vermont
n_trum : 95369
n_clinton178573
autres :41125
STATE : Vermont
Traitement :  /home/ec2-user/POTUS/data

## Alimentation Collection 4

In [32]:
list_File_col4 =list_File[18:25]
print (list_File_col4)

['2016-11-08-20-15_Hawai.txt', '2016-11-08-20-17_Wisconsin.txt', '2016-11-08-20-19_Georgie.txt', '2016-11-08-20-22_Ohio.txt', '2016-11-08-20-24_Missouri.txt', '2016-11-08-20-24_New_Hampshire.txt', '2016-11-08-20-24_Virginie.txt']


In [31]:
%%time
for file in list_File_col4:
    filename = DIR + file
    chargement(filename,DIR,"col4")
    myCurso = c.POTUS.col4.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)  

Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-15_Hawai.txt
Hawai
n_trum : 128847
n_clinton266891
autres :185426
STATE : Hawai
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-17_Wisconsin.txt
Wisconsin
n_trum : 1405284
n_clinton1382536
autres :188330
STATE : Wisconsin
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-19_Georgie.txt
Georgie
n_trum : 2089104
n_clinton1877963
autres :198338
STATE : Georgie
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-22_Ohio.txt
Ohio
n_trum : 2841005
n_clinton2394164
autres :372472
STATE : Ohio
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-24_Missouri.txt
Missouri
n_trum : 1594511
n_clinton1071068
autres :143023
STATE : Missouri
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-24_New_Hampshire.txt
New_Hampshire
n_trum : 345790
n_clinton348526
autres :49865
STATE : New_Hampshire
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-24_Virginie.txt
Virginie
n_trum : 1769443
n_c

## Alimentation Collection 5

In [35]:
list_File_col5 =list_File[25:32]
print (list_File_col5)

['2016-11-08-20-27_Iowa.txt', '2016-11-08-20-28_Delaware.txt', '2016-11-08-20-29_Arizona.txt', '2016-11-08-20-31_Louisiane.txt', '2016-11-08-20-31_Massachusetts.txt', '2016-11-08-20-31_Texas.txt', '2016-11-08-20-34_Dakota_du_Nord.txt']


In [36]:
%%time
for file in list_File_col5:
    filename = DIR + file
    chargement(filename,DIR,"col5")
    myCurso = c.POTUS.col5.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)  

Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-27_Iowa.txt
Iowa
STATE     : Iowa
n_trum    : 800983
n_clinton : 653669
autres    : 111380
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-28_Delaware.txt
Delaware
STATE     : Delaware
n_trum    : 185127
n_clinton : 265603
autres    : 20860
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-29_Arizona.txt
Arizona
STATE     : Arizona
n_trum    : 1252401
n_clinton : 1161167
autres    : 247979
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-31_Louisiane.txt
Louisiane
STATE     : Louisiane
n_trum    : 1178638
n_clinton : 780154
autres    : 70240
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-31_Massachusetts.txt
Massachusetts
STATE     : Massachusetts
n_trum    : 1090893
n_clinton : 1995196
autres    : 238957
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-31_Texas.txt
Texas
STATE     : Texas
n_trum    : 2792589
n_clinton : 3877868
autres    : 0
Traitement :  /home/ec2-user

## Alimentation Collection 6

In [58]:
list_File_col6 =list_File[32:38]
print (list_File_col6)

['2016-11-08-20-35_Tennessee.txt', '2016-11-08-20-36_Wyoming.txt', '2016-11-08-20-37_Floride.txt', '2016-11-08-20-37_Illinois.txt', '2016-11-08-20-40_Nevada.txt']


In [59]:
%%time
for file in list_File_col6:
    filename = DIR + file
    chargement(filename,DIR,"col6")
    myCurso = c.POTUS.col6.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)  

Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-35_Tennessee.txt
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-36_Wyoming.txt
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-37_Floride.txt
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-37_Illinois.txt
Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-40_Nevada.txt
CPU times: user 244 ms, sys: 68 ms, total: 312 ms
Wall time: 13min 46s


## Alimentation Collection 7

In [ ]:
list_File_col7 =list_File[38:46]
print (list_File_col7)

In [ ]:
%%time
for file in list_File_col7:
    filename = DIR + file
    chargement(filename,DIR,"col7")
    myCurso = c.POTUS.col7.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)  

In [ ]:
## Alimentation Collection 8

In [ ]:
list_File_col8 =list_File[46:51]
print (list_File_col8)

In [ ]:
%%time
for file in list_File_col8:
    filename = DIR + file
    chargement(filename,DIR,"col8")
    myCurso = c.POTUS.col8.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
    x =[]
    for i in myCurso:
        x.append(i)
    etat = file[17:-4]
    print (etat)
    prep_dico(etat,x)  

In [27]:
#%%time
#DIR = "/home/ec2-user/POTUS/data_POTUS/"
#filename =DIR + "2016-11-08-20-02_New_York.txt"
#chargement(filename,DIR)

Traitement :  /home/ec2-user/POTUS/data_POTUS/2016-11-08-20-02_New_York.txt


In [28]:
%%time
myCurso = c.POTUS.col3.aggregate([{'$group' : { '_id': '$STATE' ,'number': { '$sum' : 1} }}])

for i in myCurso:
    print(i)

myCurso1 = c.POTUS.traitement.find()  
for i in myCurso1:
    print(i)

{'number': 7787064, '_id': 'New_York'}
{'number': 804043, '_id': 'Nouveau_Mexique'}
{'traitement': '/home/ec2-user/POTUS/data_POTUS/2016-11-08-20-03_Nouveau_Mexique.txt', '_id': ObjectId('587f863373ebee0d17a1e660')}
{'traitement': '/home/ec2-user/POTUS/data_POTUS/2016-11-08-20-00_Minnesota.txt', '_id': ObjectId('587f878973ebee0e9347c59b')}
{'traitement': '/home/ec2-user/POTUS/data_POTUS/2016-11-08-20-02_New_York.txt', '_id': ObjectId('587f884973ebee0d17a1e661')}
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 14.3 s


NameError: name 'myCurso1' is not defined

In [12]:
list_File = [
'2016-11-08-20-00_Minnesota.txt'
,'2016-11-08-20-01_Alabama.txt'
,'2016-11-08-20-01_Montana.txt'
,'2016-11-08-20-01_Utah.txt'
,'2016-11-08-20-02_Caroline_du_Sud.txt'
,'2016-11-08-20-02_New_York.txt'
,'2016-11-08-20-03_Nouveau_Mexique.txt'
,'2016-11-08-20-04_Californie.txt'
,'2016-11-08-20-04_Colorado.txt'
,'2016-11-08-20-05_Connecticut.txt'
,'2016-11-08-20-08_District_de_Columbia.txt'
,'2016-11-08-20-08_Kentucky.txt'
,'2016-11-08-20-09_Caroline_du_Nord.txt'
,'2016-11-08-20-09_Vermont.txt'
,'2016-11-08-20-10_Dakota_du_Sud.txt'
,'2016-11-08-20-11_Alaska.txt'
,'2016-11-08-20-13_Arkansas.txt'
,'2016-11-08-20-13_Washington.txt'
,'2016-11-08-20-15_Hawai.txt'
,'2016-11-08-20-17_Wisconsin.txt'
,'2016-11-08-20-19_Georgie.txt'
,'2016-11-08-20-22_Ohio.txt'
,'2016-11-08-20-24_Missouri.txt'
,'2016-11-08-20-24_New_Hampshire.txt'
,'2016-11-08-20-24_Virginie.txt'
,'2016-11-08-20-27_Iowa.txt'
,'2016-11-08-20-28_Delaware.txt'
,'2016-11-08-20-29_Arizona.txt'
,'2016-11-08-20-31_Louisiane.txt'
,'2016-11-08-20-31_Massachusetts.txt'
,'2016-11-08-20-31_Texas.txt'
,'2016-11-08-20-34_Dakota_du_Nord.txt'
,'2016-11-08-20-35_Tennessee.txt'
,'2016-11-08-20-36_Wyoming.txt'
,'2016-11-08-20-37_Floride.txt'
,'2016-11-08-20-37_Illinois.txt'
,'2016-11-08-20-40_Nevada.txt'
,'2016-11-08-20-41_Idaho.txt'
,'2016-11-08-20-41_Pennsylvanie.txt'
,'2016-11-08-20-43_Virginie_Occidentale.txt'
,'2016-11-08-20-48_Maryland.txt'
,'2016-11-08-20-48_Mississippi.txt'
,'2016-11-08-20-48_Rhode_Island.txt'
,'2016-11-08-20-50_Michigan.txt'
,'2016-11-08-20-51_Oklahoma.txt'
,'2016-11-08-20-52_Nebraska.txt'
,'2016-11-08-20-56_New_Jersey.txt'
,'2016-11-08-20-57_Indiana.txt'
,'2016-11-08-20-57_Kansas.txt'
,'2016-11-08-20-58_Maine.txt'
,'2016-11-08-20-59_Oregon.txt']

In [29]:
list_File_col1 =['2016-11-08-20-00_Minnesota.txt'
,'2016-11-08-20-01_Alabama.txt'
,'2016-11-08-20-01_Montana.txt'
,'2016-11-08-20-01_Utah.txt'
,'2016-11-08-20-02_Caroline_du_Sud.txt'
,'2016-11-08-20-02_New_York.txt'
,'2016-11-08-20-03_Nouveau_Mexique.txt']


In [14]:
for file_name in list_File_col1 :
    chargement(file_name)

Traitement :  2016-11-08-20-00_Minnesota.txt
Traitement :  2016-11-08-20-01_Alabama.txt
Traitement :  2016-11-08-20-01_Montana.txt
Traitement :  2016-11-08-20-01_Utah.txt
Traitement :  2016-11-08-20-02_Caroline_du_Sud.txt
Traitement :  2016-11-08-20-02_New_York.txt
Traitement :  2016-11-08-20-03_Nouveau_Mexique.txt
Traitement :  2016-11-08-20-04_Californie.txt
Traitement :  2016-11-08-20-04_Colorado.txt
Traitement :  2016-11-08-20-05_Connecticut.txt
Traitement :  2016-11-08-20-08_District_de_Columbia.txt
Traitement :  2016-11-08-20-08_Kentucky.txt
Traitement :  2016-11-08-20-09_Caroline_du_Nord.txt
Traitement :  2016-11-08-20-09_Vermont.txt
Traitement :  2016-11-08-20-10_Dakota_du_Sud.txt
Traitement :  2016-11-08-20-11_Alaska.txt
Traitement :  2016-11-08-20-13_Arkansas.txt
Traitement :  2016-11-08-20-13_Washington.txt
Traitement :  2016-11-08-20-15_Hawai.txt
Traitement :  2016-11-08-20-17_Wisconsin.txt
Traitement :  2016-11-08-20-19_Georgie.txt
Traitement :  2016-11-08-20-22_Ohio.txt
T

In [2]:
list_File

['2016-11-08-20-00_Minnesota.txt',
 '2016-11-08-20-01_Alabama.txt',
 '2016-11-08-20-01_Montana.txt',
 '2016-11-08-20-01_Utah.txt',
 '2016-11-08-20-02_Caroline_du_Sud.txt',
 '2016-11-08-20-02_New_York.txt',
 '2016-11-08-20-03_Nouveau_Mexique.txt',
 '2016-11-08-20-04_Californie.txt',
 '2016-11-08-20-04_Colorado.txt',
 '2016-11-08-20-05_Connecticut.txt',
 '2016-11-08-20-08_District_de_Columbia.txt',
 '2016-11-08-20-08_Kentucky.txt',
 '2016-11-08-20-09_Caroline_du_Nord.txt',
 '2016-11-08-20-09_Vermont.txt',
 '2016-11-08-20-10_Dakota_du_Sud.txt',
 '2016-11-08-20-11_Alaska.txt',
 '2016-11-08-20-13_Arkansas.txt',
 '2016-11-08-20-13_Washington.txt',
 '2016-11-08-20-15_Hawai.txt',
 '2016-11-08-20-17_Wisconsin.txt',
 '2016-11-08-20-19_Georgie.txt',
 '2016-11-08-20-22_Ohio.txt',
 '2016-11-08-20-24_Missouri.txt',
 '2016-11-08-20-24_New_Hampshire.txt',
 '2016-11-08-20-24_Virginie.txt',
 '2016-11-08-20-27_Iowa.txt',
 '2016-11-08-20-28_Delaware.txt',
 '2016-11-08-20-29_Arizona.txt',
 '2016-11-08-20-3

In [31]:
%%time
#c.POTUS.election.find({"STATE": "Minnesota"}).cou
#c.POTUS.election.group({})
#c.POTUS.election.aggregate([{$group : { _id: STATE :{$STATE:"$STATE"} } }])


myCurso = c.POTUS.election.aggregate([{'$group' : { '_id': '$STATE' ,'number': { '$sum' : 1} }}])

for i in myCurso:
    print(i)


{'_id': 'Californie', 'number': 14610499}
{'_id': 'Minnesota', 'number': 2944813}
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 34.6 s


In [53]:
%%time
myCurso = c.POTUS.election.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])


CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 34.2 s


In [54]:
x =[]
for i in myCurso:
    x.append(i)
    print(i)

{'_id': {'CANDIDATE': 'Blanc', 'STATE': 'Californie'}, 'total': 428915}
{'_id': {'CANDIDATE': 'McMullin', 'STATE': 'Californie'}, 'total': 39586}
{'_id': {'CANDIDATE': 'Johnson', 'STATE': 'Californie'}, 'total': 478499}
{'_id': {'CANDIDATE': 'Stein', 'STATE': 'Californie'}, 'total': 278657}
{'_id': {'CANDIDATE': 'Clinton', 'STATE': 'Californie'}, 'total': 8753788}
{'_id': {'CANDIDATE': 'Castle', 'STATE': 'Minnesota'}, 'total': 9456}
{'_id': {'CANDIDATE': 'Autre', 'STATE': 'Minnesota'}, 'total': 41657}
{'_id': {'CANDIDATE': 'Clinton', 'STATE': 'Minnesota'}, 'total': 1367716}
{'_id': {'CANDIDATE': 'Stein', 'STATE': 'Minnesota'}, 'total': 36985}
{'_id': {'CANDIDATE': 'Autre', 'STATE': 'Californie'}, 'total': 147244}
{'_id': {'CANDIDATE': 'McMullin', 'STATE': 'Minnesota'}, 'total': 53076}
{'_id': {'CANDIDATE': 'Trump', 'STATE': 'Californie'}, 'total': 4483810}
{'_id': {'CANDIDATE': 'Johnson', 'STATE': 'Minnesota'}, 'total': 112972}
{'_id': {'CANDIDATE': 'Trump', 'STATE': 'Minnesota'}, 'tot

In [95]:
#result = pd.DataFrame(list(myCurso)

dic_election_list =[]
for elt in x :
    f = elt['total']
    e  = elt['_id']
    li = [e['STATE'],e['CANDIDATE'],f]
    dic_election_list.append(li)

df =pd.DataFrame(dic_election_list, columns=["STATE", "CANDIDATE", "ToTaL"])

print(df)

         STATE CANDIDATE    ToTaL
0   Californie     Blanc   428915
1   Californie  McMullin    39586
2   Californie   Johnson   478499
3   Californie     Stein   278657
4   Californie   Clinton  8753788
5    Minnesota    Castle     9456
6    Minnesota     Autre    41657
7    Minnesota   Clinton  1367716
8    Minnesota     Stein    36985
9   Californie     Autre   147244
10   Minnesota  McMullin    53076
11  Californie     Trump  4483810
12   Minnesota   Johnson   112972
13   Minnesota     Trump  1322951


In [63]:
%%time
etat = 'Minnesota'
myCurso = c.POTUS.col1.aggregate([{'$group' : { '_id': { 'STATE' :'$STATE', 'CANDIDATE':'$CANDIDATE'},'total': { '$sum' : 1} }}])
x =[]
for i in myCurso:
    x.append(i)


def prep_dico(nom_etat,x):

    #dic_election_list =[]
    n_autres = 0
    n_trump_vote = 0
    n_clinton_vote = 0
    for elt in x :
        
        f = elt['total']
        e  = elt['_id']
        if (e['STATE'] == nom_etat) :
            if (e['CANDIDATE'] == 'Trump') : 
                n_trump_vote = f
            else : 
                if e['CANDIDATE'] == 'Clinton' :
                    n_clinton_vote = f
                else : 
                    n_autres += f
    print('n_trum : ' + str(n_trump_vote))
    print('n_clinton' + str(n_clinton_vote))    
    print('autres :' + str(n_autres))
    print('STATE : ' + str(nom_etat))
    
    #    li = [e['STATE'],e['CANDIDATE'],f]
    #    dic_election_list.append(li)

    #df =pd.DataFrame(dic_election_list, columns=["STATE", "CANDIDATE", "ToTaL"])
    c.POTUS.synthese.insert_one(
    {
     "state": nom_etat,
     "Clinton": n_clinton_vote,
     "Trump" : n_trump_vote,
     "Other" : n_autres   })
prep_dico(etat,x)

n_trum : 1322951
n_clinton1367716
autres :254146
STATE : Minnesota
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 33.9 s


In [93]:
#result.describe()


,STATE,CANDIDATE,ToTaL
0,Californie,Blanc,428915
1,Californie,McMullin,39586
2,Californie,Johnson,478499
3,Californie,Stein,278657
4,Californie,Clinton,8753788
5,Minnesota,Castle,9456
6,Minnesota,Autre,41657
7,Minnesota,Clinton,1367716
8,Minnesota,Stein,36985
9,Californie,Autre,147244


In [77]:
COL = ["Time", "State", "Candidate"]

In [19]:
df_Minnesota = pd.read_csv("data_POTUS/2016-11-08-20-00_Minnesota.txt", sep=";", header = None, names = COL)

In [33]:
df_Minnesota["Candidate"].value_counts()

Clinton     1367716
Trump       1322951
Johnson      112972
McMullin      53076
Autre         41657
Stein         36985
Castle         9456
Name: Candidate, dtype: int64

In [25]:
df_Minnesota.tail()

,Time,State,Candidate
2944808,2016-11-08T20:00,Minnesota,Autre
2944809,2016-11-08T20:00,Minnesota,Autre
2944810,2016-11-08T20:00,Minnesota,Autre
2944811,2016-11-08T20:00,Minnesota,Autre
2944812,2016-11-08T20:00,Minnesota,Autre


In [46]:
db = c["dontexist"]
cn = db["mycollection"]

In [56]:
start_time = time.time()
many = df[2844809:2944809].to_dict(orient='records')
cn.insert_many(many)
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.296091079711914 seconds ---


In [81]:
len(many)

100000

In [61]:
c

MongoClient(host=['172.31.31.101:27017', '172.31.31.100:27017', '172.31.31.102:27017'], document_class=dict, tz_aware=False, connect=True, replicaset='replica-set')

In [49]:
cn.count()

1500026

In [57]:
cn.find({"Candidate": "Autre"}).count()

41653

In [89]:
len(df)

2944813

In [45]:
c.dontexist.collection_names()

['mycollection', 'system.indexes']

In [49]:
db = c["dontexist"]

In [50]:
db.collection_names()


['mycollection', 'system.indexes']

In [54]:
cn = db["mycollection"]

In [67]:
print(collection.find_one())

{'item': 'box', '_id': ObjectId('58769be76ee75c8173207f03'), 'qty': 20.0}


In [64]:
cn.insert_one(
    {"Time":"2016-11-08T20:00",
     "State":"Minnesota",
     "Candidate":"Clinton"})

In [80]:
cn.insert_many(
    [{"Time":"2016-11-08T20:00",
      "State":"Minnesota","Candidate":"Clinton"},
     {"Time":"2016-11-08T20:00",
      "State":"Minnesota","Candidate":"Clinton"}])

In [88]:
cn.insert_many(many)

In [62]:
one

'[{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"}]'

In [85]:
many

'[{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-08T20:00","State":"Minnesota","Candidate":"Clinton"},{"Time":"2016-11-0